In [ ]:
import numpy as np
import datetime

from btgym import BTgymEnv, BTgymDataset
from btgym.datafeed import BTgymRandomDataDomain, BTgymSequentialDataDomain

from logbook import WARNING, INFO, DEBUG

- BTGym data is basically discrete timeflow of equitype records. For the sake of
defining episodic MDP over such data and setting formal problem objective it should
be somehow structured. 

- This notebook is brief introduction to API realisation of formal definitions introduced in Section 1 (Data) here: https://github.com/Kismuz/btgym/docs/papers/btgym_formalism_draft.pdf

In [ ]:
# Make data domain - top-level data structure:
domain = BTgymRandomDataDomain(
    filename='../examples/data/DAT_ASCII_EURUSD_M1_2016.csv',
    trial_params=dict(
        start_weekdays={0, 1, 2, 3, 4, 5, 6},
        sample_duration={'days': 10, 'hours': 0, 'minutes': 0},
        start_00=True,
        time_gap={'days': 5, 'hours': 0},
        test_period={'days': 2, 'hours': 0, 'minutes': 0},
    ),
    episode_params=dict(
        start_weekdays={0, 1, 2, 3, 4, 5, 6},
        sample_duration={'days': 0, 'hours': 23, 'minutes': 55},
        start_00=False,
        time_gap={'days': 0, 'hours': 10},
    ),
    log_level=INFO, # Set to DEBUG to see more output
)

##### Manually perfom sampling cycle:

- Prepare domain, it's essential for stateful classes such as BTgymSequentialDataDomain:

In [ ]:
domain.reset()

##### Sample trial subset from domain:

In [ ]:
trial= domain.sample()
trial.reset()

- Note that during real BTgym operation domain instance is held by `btgym_data_server`;

- Different Trial sample is sent to every environment instance, so corresponding `btgym_server` can sample multiple episodes until agent decides to request another trial via Gym API by calling env.reset() 
with kwarg `new_trial`=True.


- Episodes can be sampled from Trial `train` or `test` subsets by passing kwarg 'type' to reset method, e.g:       

    - env.reset(new_trial=False, type=1) will ask for test episode from same trial;
    
    - env.reset(new_trial=True, type=0) asks for train episode from new trial (... means requset is passed from API shell to `btgym_server` wich itself request new episode from `data_server` and than samples episode from freshly obtained trial).
    
        
- To be implemented soon: pass kwargs `b_alpha` and `b_beta` to get skewed train data sampling.

##### Sample episode from train trial interval:

In [ ]:
episode = trial.sample(sample_type=0)

##### Now print whole path:

In [ ]:
print('Got instance of: {}\nholding data: {}\nmetadata: {}'.
      format(type(domain), domain.filename, domain.metadata))

print('  |\nsample()\n  |')

print('got instance of: {}\nholding data: {}\nmetadata: {}'.
      format(type(trial), trial.filename, trial.metadata))

print('  |\nsample()\n  |')

print('got instance of: {}\nholding data: {}\nmetadata: {}'.
      format(type(episode), episode.filename, episode.metadata))

print('  |\nto_btfeed()\n  |')

print('got instance of: {},\n...wich ready to be fed to bt.Cerebro'.format(type(episode.to_btfeed())))

##### Note:
- BTgymDataSet class used in most examples is simply special case where we set Trial=Domain by definition.
- This nested data structure is intended mostly for upcoming implementation of meta-learning and guided-policy-search algorithms.
- using kwargs in .reset() method is Gym API extension. Nevermind.